In [1]:
import pandas as pd
import numpy as np


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# https://github.com/jamesqo/gun-violence-data
df = pd.read_csv('./gun_violence_dataset/gun_violence_dataset.csv')
df.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


Veamos el número de registros vacíos por cada columna:

In [4]:
print("El dataset contiene un total de {} filas".format(len(df)))


df.isna().sum()


El dataset contiene un total de 239677 filas


incident_id                         0
date                                0
state                               0
city_or_county                      0
address                         16497
n_killed                            0
n_injured                           0
incident_url                        0
source_url                        468
incident_url_fields_missing         0
congressional_district          11944
gun_stolen                      99498
gun_type                        99451
incident_characteristics          326
latitude                         7923
location_description           197588
longitude                        7923
n_guns_involved                 99451
notes                           81017
participant_age                 92298
participant_age_group           42119
participant_gender              36362
participant_name               122253
participant_relationship       223903
participant_status              27626
participant_type                24863
sources     

Eliminamos las columnas con mayor cantidad de valores nulos (participant_relationship, location_description, participant_name), que son prescindibles o contienen información personal que no nos interesa. 

Eliminamos también incident_url_fields_missing que siempre tiene valor False y las columnas que tienen las url de las noticias a excepción de incident_url que no tiene ningún valor faltante (sources, source_url). 

Por último, prescindimos también de las columnas state_house_district y state_senate_district, a favor de congressional_district, que es la que menor número de valores nulos tiene y porque las tres columnas se refieren a lo mismo: la entidad territorial que elige a un solo miembro del congreso.

In [5]:
del df["participant_relationship"], df["location_description"], df["participant_name"], df["incident_url_fields_missing"], df["sources"], df["source_url"], df["state_house_district"], df["state_senate_district"]
df.isna().sum()

incident_id                     0
date                            0
state                           0
city_or_county                  0
address                     16497
n_killed                        0
n_injured                       0
incident_url                    0
congressional_district      11944
gun_stolen                  99498
gun_type                    99451
incident_characteristics      326
latitude                     7923
longitude                    7923
n_guns_involved             99451
notes                       81017
participant_age             92298
participant_age_group       42119
participant_gender          36362
participant_status          27626
participant_type            24863
dtype: int64

Una vez eliminadas las columnas con mayor número de valores faltantes y aquellas que no nos son de interés, procedemos a dividir el dataset en tres subdatasets, cada uno de los cuales será tratado en cuanto a valores nulos y faltantes; en lugar de hacer esto antes de dividirlo pues se perdería información.

In [6]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55226 entries, 4 to 239676
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   incident_id               55226 non-null  int64  
 1   date                      55226 non-null  object 
 2   state                     55226 non-null  object 
 3   city_or_county            55226 non-null  object 
 4   address                   55226 non-null  object 
 5   n_killed                  55226 non-null  int64  
 6   n_injured                 55226 non-null  int64  
 7   incident_url              55226 non-null  object 
 8   congressional_district    55226 non-null  float64
 9   gun_stolen                55226 non-null  object 
 10  gun_type                  55226 non-null  object 
 11  incident_characteristics  55226 non-null  object 
 12  latitude                  55226 non-null  float64
 13  longitude                 55226 non-null  float64
 14  n_gun

In [7]:
df.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_status,participant_type
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...
10,491674,2013-01-23,Tennessee,Chattanooga,1501 Dodds Ave,1,3,http://www.gunviolencearchive.org/incident/491674,3.0,0::Unknown,0::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",35.0221,-85.2697,1.0,19 yr. old male dies; 3 inured; shooting on Do...,0::19,0::Adult 18+,0::Male||1::Male||2::Male||3::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...
11,479413,2013-01-25,Missouri,Saint Louis,W Florissant Ave and Riverview Blvd,1,3,http://www.gunviolencearchive.org/incident/479413,1.0,0::Unknown,0::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",38.7067,-90.2494,1.0,"38.706732, -90.249375",0::28,0::Adult 18+,0::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...
12,479561,2013-01-26,Louisiana,Charenton,1000 block of Flat Town Road,2,3,http://www.gunviolencearchive.org/incident/479561,3.0,0::Unknown,0::Shotgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",29.8816,-91.5251,1.0,Ofc. hailed from Chitimacha Tribe of Louisiana...,3::78||4::48,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Killed||1::Injured||2::Injured||3::Killed||...,0::Victim||1::Victim||2::Victim||3::Victim||4:...
16,479580,2013-02-03,California,Yuba (county),5800 block of Poplar Avenue,1,3,http://www.gunviolencearchive.org/incident/479580,3.0,0::Unknown,0::9mm,"Shot - Wounded/Injured||Shot - Dead (murder, a...",39.1236,-121.5830,1.0,"perps have gang affiliation, but unclear as to...",0::20||4::25||5::18||6::19,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Female||4::Male||5::Male|...,0::Killed||1::Injured||2::Injured||3::Injured|...,0::Victim||1::Victim||2::Victim||3::Victim||4:...


# Tratamiento de datos de tipo objeto
Entendemos como una columna de tipo objeto aquellas que siguen la sintaxis i::value1||i+1::value2, donde i es un número mayor o igual que 0 y value es un string que indica su valor

In [8]:
example = df['participant_age'][4]
example

'0::18||1::46||2::14||3::47'

## Funciones
A continuación, se definen una serie de funciones que pueden ser útiles para preprocesar columnas de este tipo, también podría estudiarse el definir las siguientes funciones:
- Valor mínimo
- Valor máximo
- Desviación estándar

In [9]:
# Devuelve el conjunto de objectos de una celda
def get_elements_as_list(cell):
#     print(cell)
    if('||' in cell):
        return cell.split('||')
    else:
        return cell.split('|')

In [10]:
# Devuelve el número de elementos
def get_num_elements_of_object(cell):
    elements = get_elements_as_list(cell)
    return len(elements)

In [11]:
# Devuelve un map donde key=i y value=value
def get_elements(cell):
    res = {}
    elements = get_elements_as_list(cell)
    i = 0
    for element in elements:
#         print(element)
        if '::' in element:
            value = element.split('::')[1]
        else:
            value = element.split(':')[1]
        res[i] = value
        i = i + 1
    return res

In [12]:
# Devuelve los elementos (get_elements) como números
def get_elements_as_numbers(cell):
    res = get_elements(cell)
    
    # Convert to number
    for i in range(0, len(res)):
        res[i] = float(res[i])
        
    return res

In [13]:
# Calcula el valor medio de una columna con objetos de tipo número
def compute_average_value(cell):
    elements = get_elements_as_numbers(cell)
    ac = 0.0
    
    for i in range(0, len(elements)):
        ac = ac + elements[i]
        
    return ac/len(elements)

In [14]:
# Get number of victims and suspects
def get_number_of_value(cell, value):
    elements = get_elements(cell)
    
    n_values = 0
    
    for i in range(0, len(elements)):
        if(elements[i].lower() == value.lower()):
            n_values = n_values + 1
    
    return n_values

In [15]:
# Returns true if an object contains part of a string
def get_contains(cell, value):
    
    elements = get_elements_as_list(cell)
    
    for i in range(0, len(elements)):
        if value in elements[i].lower():
            return True
    
    return False

In [16]:
# Returns true if an object contains exactly a string
def get_contains_strict(cell, value):
    
    elements = get_elements_as_list(cell)
    
    for i in range(0, len(elements)):
        if value == elements[i].lower():
            return True
    
    return False

In [17]:
# Returns true if an object contains exactly one element of a list of strings
def get_contains_strict_multiple(cell, values):
    
    res = False
    for value in values:
        res = get_contains_strict(cell, value)
        
        if res:
            return res
    
    return res

In [18]:
# Imprime un valor si este no aparece en la lista pasada por parámetro, se usa para encontrar valores no contemplados en parámetros de tipo enumerado
def print_not_considered(cell, values, simple_list=False):
    
    if simple_list:
        elements = get_elements_as_list(cell)
    else:
        elements = get_elements(cell)
        
#     print(elements)
    
    for i in range(0, len(elements)):
        if elements[i].lower() not in values:
#             print('Values: {}'.format(values))
#             print('Element:{}'.format(elements[i]))
            print(elements[i])
    

In [19]:
# Similar to print_not_considered, but with substrings
def print_not_considered_contains(cell, values):
    
    elements = get_elements_as_list(cell)
             
            
    for i in range(0, len(elements)):
        contains = False
        for value in values:
            if value in elements[i].lower():
                contains = True
        
        if not contains:
            print(elements[i])
            

## Tests

In [20]:
splitted = get_elements_as_list(example)
splitted

['0::18', '1::46', '2::14', '3::47']

In [21]:
length = get_num_elements_of_object(example)
length

4

In [22]:
elements = get_elements(example)
elements

{0: '18', 1: '46', 2: '14', 3: '47'}

In [23]:
numbers = get_elements_as_numbers(example)
numbers

{0: 18.0, 1: 46.0, 2: 14.0, 3: 47.0}

In [24]:
average = compute_average_value(example)
average

31.25

# Creación de nuevas columnas

## 1.- Edad media de los participantes

In [25]:
df['average_age'] = df['participant_age'].apply(compute_average_value)
# df.head()

## 2.- Número de víctimas

In [26]:
# Antes de nada, comprobamos que no existen más valores en la columna participant_type, exceptuando Victim y subject-suspect
def print_not_considered_participant_type(cell):
    return print_not_considered(cell, ['victim', 'subject-suspect'])

df['participant_type'].apply(print_not_considered_participant_type)

4         None
10        None
11        None
12        None
16        None
          ... 
239654    None
239664    None
239671    None
239675    None
239676    None
Name: participant_type, Length: 55226, dtype: object

In [27]:
def get_num_victims(cell):
    return get_number_of_value(cell, 'Victim')

In [28]:
df['n_victims'] = df['participant_type'].apply(get_num_victims)
# df.head()

## 3.- Número de sospechosos 

In [29]:
def get_num_suspects(cell):
    return get_number_of_value(cell, 'Subject-Suspect')

In [30]:
df['n_suspects'] = df['participant_type'].apply(get_num_suspects)
# df.head()

## 4.- Convertir tipos de incidentes en booleans

In [31]:
# ['Shot - Wounded/Injured',
# 'Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one location)',
# 'Possession (gun(s) found during commission of other crimes)',
# 'Possession of gun by felon or prohibited person']

Categorías:

- Domestic Violence
- Drive-by
- Gang involvement
- Home Invasion
- Institution/Group/Business
- Mass Shooting
- Murder
- Officer Involved
- Possession
- Shot
- Spree Shooting
- Suicide

In [32]:
def get_incident_types():
    return [
                                    'Domestic Violence'.lower(),
                                        'Drive-by'.lower(),
                                        'Gang involvement'.lower(),
                                        'Home Invasion'.lower(),
                                        'Institution/Group/Business'.lower(),
                                        'Shooting'.lower(),
                                        'Murder'.lower(),
                                        'Officer Involved'.lower(),
                                        'Possession'.lower(),
                                        'Shot'.lower(),
                                        'Suicide'.lower(),
                                    'Drug involvement'.lower(),
                                    'Stolen/Illegally owned gun'.lower(),
                                    'Bar/club incident'.lower(),
                                    'Sex crime'.lower(),
                                    'School Incident'.lower(),
                                    'Kidnapping/abductions/hostage'.lower(),
                                    'Defensive Use'.lower(),
                                    'Car-jacking'.lower(),
                                    'Armed robbery'.lower(),
                                    'Hate crime'.lower(),
                                    'Child'.lower(),
                                    'Accidental/Negligent Discharge'.lower(),
                                    'Brandishing/flourishing/open carry/lost/found'.lower(),
                                    'Gun(s) stolen'.lower(),
                                    'Guns stolen'.lower(), # Group with the previous one
                                    'Road rage'.lower(),
                                    'Under the influence of alcohol or drugs'.lower(),
                                    'BB/Pellet/Replica gun'.lower(),
                                    'Cleaning gun'.lower(),
                                    'Implied Weapon'.lower(),
                                    'House party'.lower(),
                                    'ATF/LE Confiscation/Raid/Arrest'.lower(),
                                    'Gun at school'.lower(),
                                    'Thought gun was unloaded'.lower(),
                                    'Hunting accident'.lower(),
                                    'stolen gun'.lower(),
                                    'Police Targeted'.lower(),
                                    'Pistol-whipping'.lower(),
                                    'Playing with gun'.lower(),
                                    'Shootout'.lower(),
                                    'Unlawful purchase/sale'.lower(),
                                    'Non-Aggression'.lower(),
                                    'Gun shop robbery or burglary'.lower(),
                                    'Concealed Carry License'.lower(),
                                    'Assault weapon'.lower(),
                                    'LOCKDOWN/ALERT'.lower(),
                                    'TSA Action'.lower(),
                                    'Terrorism'.lower(),
                                    'Ghost gun'.lower(),
                                    'Political Violence'.lower(),
                                    'Mistaken ID'.lower(),
                                    'NAV'.lower(),
                                    'Gun buy back action'.lower()
                                ]

In [33]:
# Antes de nada, comprobamos que no existen más valores en la columna participant_type, exceptuando Victim y subject-suspect
def print_not_considered_incident_characteristics(cell):
    incident_types = get_incident_types()
    return print_not_considered_contains(cell, 
                                incident_types
                               )

df['incident_characteristics'].apply(print_not_considered_incident_characteristics)

4         None
10        None
11        None
12        None
16        None
          ... 
239654    None
239664    None
239671    None
239675    None
239676    None
Name: incident_characteristics, Length: 55226, dtype: object

In [34]:
for incident_type in get_incident_types():
    df[incident_type] = df['incident_characteristics'].apply(get_contains, value=incident_type)

In [35]:
df.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_status,participant_type,average_age,n_victims,n_suspects,domestic violence,drive-by,gang involvement,home invasion,institution/group/business,shooting,murder,officer involved,possession,shot,suicide,drug involvement,stolen/illegally owned gun,bar/club incident,sex crime,school incident,kidnapping/abductions/hostage,defensive use,car-jacking,armed robbery,hate crime,child,accidental/negligent discharge,brandishing/flourishing/open carry/lost/found,gun(s) stolen,guns stolen,road rage,under the influence of alcohol or drugs,bb/pellet/replica gun,cleaning gun,implied weapon,house party,atf/le confiscation/raid/arrest,gun at school,thought gun was unloaded,hunting accident,stolen gun,police targeted,pistol-whipping,playing with gun,shootout,unlawful purchase/sale,non-aggression,gun shop robbery or burglary,concealed carry license,assault weapon,lockdown/alert,tsa action,terrorism,ghost gun,political violence,mistaken id,nav,gun buy back action
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,31.25,3,1,True,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
10,491674,2013-01-23,Tennessee,Chattanooga,1501 Dodds Ave,1,3,http://www.gunviolencearchive.org/incident/491674,3.0,0::Unknown,0::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",35.0221,-85.2697,1.0,19 yr. old male dies; 3 inured; shooting on Do...,0::19,0::Adult 18+,0::Male||1::Male||2::Male||3::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,19.00,4,1,False,False,False,False,False,True,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
11,479413,2013-01-25,Missouri,Saint Louis,W Florissant Ave and Riverview Blvd,1,3,http://www.gunviolencearchive.org/incident/479413,1.0,0::Unknown,0::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",38.7067,-90.2494,1.0,"38.706732, -90.249375",0::28,0::Adult 18+,0::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,28.00,4,1,False,True,True,False,False,True,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
12,479561,2013-01-26,Louisiana,Charenton,1000 block of Flat Town Road,2,3,http://www.gunviolencearchive.org/incident/479561,3.0,0::Unknown,0::Shotgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",29.8816,-91.5251,1.0,Ofc. hailed from Chitimacha Tribe of Louisiana...,3::78||4::48,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Killed||1::Injured||2::Injured||3::Killed||...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,63.00,4,1,False,False,F

## 5.- Convertir gun_stolen a boolean
True si alguna de las armas ha sido robada, False en el caso contrario. Borrar esta columna, ya forma parte de incident-type

In [36]:
def print_not_considered_gun_stolen(cell):
    return print_not_considered(cell, ['unknown', 'stolen', 'not-stolen'])

df['gun_stolen'].apply(print_not_considered_gun_stolen)

4         None
10        None
11        None
12        None
16        None
          ... 
239654    None
239664    None
239671    None
239675    None
239676    None
Name: gun_stolen, Length: 55226, dtype: object

## 6.- Crear boolean minors_involved

In [37]:
def print_not_considered_participant_age_group(cell):
    return print_not_considered(cell, ['adult 18+', 'teen 12-17', 'child 0-11'])

df['participant_age_group'].apply(print_not_considered_participant_age_group)

4         None
10        None
11        None
12        None
16        None
          ... 
239654    None
239664    None
239671    None
239675    None
239676    None
Name: participant_age_group, Length: 55226, dtype: object

In [38]:
df.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_status,participant_type,average_age,n_victims,n_suspects,domestic violence,drive-by,gang involvement,home invasion,institution/group/business,shooting,murder,officer involved,possession,shot,suicide,drug involvement,stolen/illegally owned gun,bar/club incident,sex crime,school incident,kidnapping/abductions/hostage,defensive use,car-jacking,armed robbery,hate crime,child,accidental/negligent discharge,brandishing/flourishing/open carry/lost/found,gun(s) stolen,guns stolen,road rage,under the influence of alcohol or drugs,bb/pellet/replica gun,cleaning gun,implied weapon,house party,atf/le confiscation/raid/arrest,gun at school,thought gun was unloaded,hunting accident,stolen gun,police targeted,pistol-whipping,playing with gun,shootout,unlawful purchase/sale,non-aggression,gun shop robbery or burglary,concealed carry license,assault weapon,lockdown/alert,tsa action,terrorism,ghost gun,political violence,mistaken id,nav,gun buy back action
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,31.25,3,1,True,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
10,491674,2013-01-23,Tennessee,Chattanooga,1501 Dodds Ave,1,3,http://www.gunviolencearchive.org/incident/491674,3.0,0::Unknown,0::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",35.0221,-85.2697,1.0,19 yr. old male dies; 3 inured; shooting on Do...,0::19,0::Adult 18+,0::Male||1::Male||2::Male||3::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,19.00,4,1,False,False,False,False,False,True,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
11,479413,2013-01-25,Missouri,Saint Louis,W Florissant Ave and Riverview Blvd,1,3,http://www.gunviolencearchive.org/incident/479413,1.0,0::Unknown,0::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",38.7067,-90.2494,1.0,"38.706732, -90.249375",0::28,0::Adult 18+,0::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,28.00,4,1,False,True,True,False,False,True,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
12,479561,2013-01-26,Louisiana,Charenton,1000 block of Flat Town Road,2,3,http://www.gunviolencearchive.org/incident/479561,3.0,0::Unknown,0::Shotgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",29.8816,-91.5251,1.0,Ofc. hailed from Chitimacha Tribe of Louisiana...,3::78||4::48,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Killed||1::Injured||2::Injured||3::Killed||...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,63.00,4,1,False,False,F

In [40]:
df['participant_age_group'].apply(get_contains_strict_multiple, values=['Teen 12-17', 'Child 0-11'])

4         False
10        False
11        False
12        False
16        False
          ...  
239654    False
239664    False
239671    False
239675    False
239676    False
Name: participant_age_group, Length: 55226, dtype: bool